<a href="https://colab.research.google.com/github/maeda/braincancer/blob/master/MRI_tumor_segmentation_Brats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install medpy

In [10]:
# Dataset BraTS 2019 training em https://drive.google.com/drive/folders/1fdHMkpGIuMNJTZioDGskxmueIkFF_Ssn?usp=sharing
# Para funcionar sem modificar o path abaixo, copie esta pasta no seu Google Drive em <raiz>/datasets/
# Ou modifique o path abaixo para o seu diretório, onde /content/drive/My Drive/ é a raiz do seu Google Drive

from google.colab import drive
drive.mount('/content/drive')
brats2019_path = '/content/drive/My Drive/datasets/MICCAI_BraTS_2019_Data_Training' 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%tensorflow_version 1.x
import tensorflow as tf
# from tensorflow.contrib.keras.python.keras.layers import *
from keras.layers import *


def Conv3DWithBN(x, filters, ksize, strides, name, padding='same', dilation_rate=1, center=True, scale=True, decay=0.99):
    x = Conv3D(filters=filters, kernel_size=ksize, strides=strides, padding=padding, dilation_rate=dilation_rate,
                        use_bias=False, kernel_initializer='he_normal', name=name+'_conv')(x)
    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn')(x)
    x = Activation('relu', name=name+'_relu')(x)
    return x


def Conv2DWithBN(x, filters, ksize, strides, name, padding='same', dilation_rate=1, center=True, scale=True, decay=0.99):
    x = Conv2D(filters=filters, kernel_size=ksize, strides=strides, padding=padding, dilation_rate=dilation_rate,
                        use_bias=False, kernel_initializer='he_normal', name=name+'_conv')(x)
    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn')(x)
    x = Activation('relu', name=name+'_relu')(x)
    return x


def Conv1DWithBN(x, filters, ksize, strides, name, padding='same', dilation_rate=1, center=True, scale=True, decay=0.99):
    x = Conv1D(filters=filters, kernel_size=ksize, strides=strides, padding=padding, dilation_rate=dilation_rate,
               use_bias=False, kernel_initializer='he_normal', name=name+'_conv')(x)
    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn')(x)
    x = Activation('relu', name=name+'_relu')(x)
    return x


def DenseWithBN(x, units, name, kernel_regularizer=None, center=True, scale=True, decay=0.99):
    x = Dense(units=units, use_bias=False, kernel_regularizer=kernel_regularizer, name=name+'_weight')(x)
    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bias')(x)
    x = Activation('relu', name=name+'_relu')(x)
    return x


def ResNetUnit2D(x, filters, ksize, name, end=False, center=True, scale=True, decay=0.99):
    identity = x

    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn_1')(x)
    x = Activation('relu', name=name+'_relu_1')(x)
    x = Conv2D(filters, kernel_size=ksize, strides=1, padding='same', kernel_initializer='he_normal', name=name+'_conv_1')(x)

    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn_2')(x)
    x = Activation('relu', name=name+'_relu_2')(x)
    x = Conv2D(filters, kernel_size=ksize, strides=1, padding='same', kernel_initializer='he_normal', name=name+'_conv2')(x)

    x = add([x, identity])
    if end:
        x = BatchNormalization(center=center, scale=scale, momentum=decay)(x)
        x = Activation('relu')(x)
    return x


def ResNetUnitIncreasingDims2D(x, filters, ksize, strides, name, begin=False, center=True, scale=True, decay=0.99):
    identity = x

    if not begin:
        x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn_1')(x)
        x = Activation('relu', name=name+'_relu_1')(x)
    x = Conv2D(filters, kernel_size=ksize, strides=strides[0], padding='same', kernel_initializer='he_normal', use_bias=False, name=name+'_conv_1')(x)

    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn_2')(x)
    x = Activation('relu', name=name+'_relu_2')(x)
    x = Conv2D(filters, kernel_size=ksize, strides=strides[1], padding='same', kernel_initializer='he_normal', use_bias=False, name=name+'_conv_2')(x)

    identity = Conv2D(filters, kernel_size=1, strides=strides[0], padding='same', kernel_initializer='he_normal', name=name+'_conv_identity')(identity)
    x = add([x, identity])
    return x


def ResNetUnit1D(x, filters, ksize, name, end=False, center=True, scale=True, decay=0.99):
    identity = x

    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn_1')(x)
    x = Activation('relu', name=name+'_relu_1')(x)
    x = Conv1D(filters, kernel_size=ksize, strides=1, padding='same', kernel_initializer='he_normal', use_bias=False, name=name+'_conv_1')(x)

    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn_2')(x)
    x = Activation('relu', name=name+'_relu_2')(x)
    x = Conv1D(filters, kernel_size=ksize, strides=1, padding='same', kernel_initializer='he_normal', use_bias=False, name=name+'_conv_2')(x)

    x = add([x, identity])
    if end:
        x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn_3')(x)
        x = Activation('relu', name=name+'_relu_3')(x)
    return x


def ResNetUnitIncreasingDims1D(x, filters, ksize, strides, name, begin=False, center=True, scale=True, decay=0.99):
    '''
    ResNet unit without BottleNeck. 2 layers
    :param x:
    :param filters:
    :param ksize:
    :param strides: list with 2 elements, stride for each layer
    :param begin:
    :return:
    '''

    identity = x

    if not begin:
        x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn_1')(x)
        x = Activation('relu', name=name+'_relu_1')(x)
    x = Conv1D(filters, kernel_size=ksize, strides=strides[0], padding='same', kernel_initializer='he_normal', use_bias=False, name=name+'_conv_1')(x)

    x = BatchNormalization(center=center, scale=scale, momentum=decay, name=name+'_bn_2')(x)
    x = Activation('relu', name=name+'_relu_2')(x)
    x = Conv1D(filters, kernel_size=ksize, strides=strides[1], padding='same', kernel_initializer='he_normal', use_bias=False, name=name+'_conv_2')(x)

    identity = Conv1D(filters, kernel_size=1, strides=strides[0], padding='same', kernel_initializer='he_normal', name=name+'_conv_identity')(identity)
    x = add([x, identity])
    return x


def ContextualAvgPooling(x, ksizes, strides):
    '''
    Concatenate input with pooling result
    :param x:
    :param ksizes:
    :param strides:
    :return:
    '''
    out = None
    for ks in ksizes:
        x_pooled = AvgPool1D(pool_size=ks, strides=strides, padding='same')
        if out is None:
            out = x_pooled
        else:
            out = concatenate([out, x_pooled], axis=-1)
    x = concatenate([x, out], axis=-1)
    return x


def ContextualAtrousConv1D(x, filters, ksize, strides, dilation_rates, name):
    """
    Retrieve contextual information with Atrous Convolution
    :param x:
    :param filters:
    :param ksize:
    :param strides:
    :param dilation_rates:
    :return:
    """
    concat = x
    for dr in dilation_rates:
        x_atrous = Conv1DWithBN(x, filters=filters, ksize=ksize, strides=strides, dilation_rate=dr, name=name+'a_conv_s'+str(dr))
        concat = concatenate([concat, x_atrous], axis=-1)
    concat = Conv1DWithBN(concat, filters=256, ksize=ksize, strides=strides, name=name+'atrou_post_conv1')
    return concat


def ContextualAtrousConv3D(x, filters, ksize, strides, dilation_rates, name):
    """
    Retrieve contextual information with Atrous Convolution
    :param x:
    :param filters:
    :param ksize:
    :param strides:
    :param dilation_rates:
    :return:
    """
    concat = None
    for dr in dilation_rates:
        x_atrous = Conv3DWithBN(x, filters=filters, ksize=ksize, strides=strides, dilation_rate=dr, name=name+'a_conv_s'+str(dr))
        if concat is None:
            concat = x_atrous
        else:
            concat = concatenate([concat, x_atrous], axis=-1)
    concat = Conv3DWithBN(concat, filters=filters, ksize=ksize, strides=strides, name=name+'atrou_post_conv1')
    return concat


def SharedAtrousConv1D(x, SharedConvs, PostConv):
    concat = x
    for SharedConv in SharedConvs:
        x_atrous = SharedConv(x)
        x_atrous = BatchNormalization()(x_atrous)
        x_atrous = Activation('relu')(x_atrous)
        concat = concatenate([concat, x_atrous], axis=-1)
    concat = PostConv(concat)
    concat = BatchNormalization()(concat)
    concat = Activation('relu')(concat)
    return concat


def densenet_block3d(x, k, rep):
    dense_input = x
    for i in range(rep):
        x_dense = Conv3D(filters=k, kernel_size=3, strides=1, padding='same', activation='relu')(dense_input)
        dense_input = concatenate([dense_input, x_dense])
    return dense_input


def DenseNetTransit(x, rate=1, name=None):
    if rate != 1:
        out_features = x.get_shape().as_list()[-1] * rate
        x = BatchNormalization(center=True, scale=True, name=name + '_bn')(x)
        x = Activation('relu', name=name + '_relu')(x)
        x = Conv3D(filters=out_features, kernel_size=1, strides=1, padding='same', kernel_initializer='he_normal',
                   use_bias=False, name=name + '_conv')(x)
    x = AveragePooling3D(pool_size=2, strides=2, padding='same')(x)
    return x


def DenseNetUnit3D(x, growth_rate, ksize, rep, bn_decay=0.99, name=None):
    for i in range(rep):
        concat = x
        x = BatchNormalization(center=True, scale=True, momentum=bn_decay, name=name+'_bn_rep_'+str(i))(x)
        x = Activation('relu')(x)
        x = Conv3D(filters=growth_rate, kernel_size=ksize, padding='same',
                   kernel_initializer='glorot_normal', use_bias=False, name=name+'_conv_rep_'+str(i))(x)
        x = concatenate([concat, x])
    return x


class BilinearUpsampling3D(Layer):
    """
    Wrapping 1D BilinearUpsamling as a Keras layer
    Input: 3D Tensor (batch, dim, channels)
    """
    def __init__(self, size, **kwargs):
        self.size = size
        super(BilinearUpsampling3D, self).__init__(**kwargs)

    def build(self, input_shape):
        super(BilinearUpsampling3D,self).build(input_shape)

    def call(self, x, mask=None):
        x = tf.expand_dims(x, axis=2)
        x = tf.image.resize_bilinear(x, [self.size, 1])
        x = tf.squeeze(x, axis=2)
        return x

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.size, input_shape[2])


# def SharedAtrousConv1D(x, SharedConvs, PostConv):
#     concat = None
#     for SharedConv in SharedConvs:
#         x_atrous = SharedConv(x)
#         x_atrous = BatchNormalization()(x_atrous)
#         x_atrous = Activation('relu')(x_atrous)
#         if concat is None:
#             concat = x_atrous
#         else:
#             concat = concatenate([concat, x_atrous], axis=-1)
#     concat = PostConv(concat)
#     concat = BatchNormalization()(concat)
#     concat = Activation('relu')(concat)
#     return concat


class BilinearUpsampling1D(Layer):
    """
    Wrapping 1D BilinearUpsamling as a Keras layer
    Input: 3D Tensor (batch, dim, channels)
    """
    def __init__(self, size, **kwargs):
        self.size = size
        super(BilinearUpsampling1D, self).__init__(**kwargs)

    def build(self, input_shape):
        super(BilinearUpsampling1D,self).build(input_shape)

    def call(self, x, mask=None):
        x = tf.expand_dims(x, axis=2)
        x = tf.image.resize_bilinear(x, [self.size, 1])
        x = tf.squeeze(x, axis=2)
        return x

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.size, input_shape[2])





In [0]:
# from tf_layers import *


def PlainCounterpart(input, name):

    x = Conv3DWithBN(input, filters=24, ksize=3, strides=1, padding='same', name=name + '_conv_15rf_1x')
    x = Conv3DWithBN(x, filters=36, ksize=3, strides=1, padding='same', name=name + '_conv_15rf_2x')
    x = Conv3DWithBN(x, filters=48, ksize=3, strides=1, padding='same', name=name + '_conv_15rf_3x')
    x = Conv3DWithBN(x, filters=60, ksize=3, strides=1, padding='same', name=name + '_conv_15rf_4x')
    x = Conv3DWithBN(x, filters=72, ksize=3, strides=1, padding='same', name=name + '_conv_15rf_5x')
    x = Conv3DWithBN(x, filters=84, ksize=3, strides=1, padding='same', name=name + '_conv_15rf_6x')
    x = Conv3DWithBN(x, filters=96, ksize=3, strides=1, padding='same', name=name + '_conv_15rf_7x')

    out_15rf = x

    x = Conv3DWithBN(x, filters=108, ksize=3, strides=1, padding='same', name=name + '_conv_27rf_1x')
    x = Conv3DWithBN(x, filters=120, ksize=3, strides=1, padding='same', name=name + '_conv_27rf_2x')
    x = Conv3DWithBN(x, filters=132, ksize=3, strides=1, padding='same', name=name + '_conv_27rf_3x')
    x = Conv3DWithBN(x, filters=144, ksize=3, strides=1, padding='same', name=name + '_conv_27rf_4x')
    x = Conv3DWithBN(x, filters=156, ksize=3, strides=1, padding='same', name=name + '_conv_27rf_5x')
    x = Conv3DWithBN(x, filters=168, ksize=3, strides=1, padding='same', name=name + '_conv_27rf_6x')

    out_27rf = x

    return out_15rf, out_27rf


def BraTS2ScaleDenseNetConcat(input, name):

    x = Conv3D(filters=24, kernel_size=3, strides=1, padding='same', name=name+'_conv_init')(input)
    x = DenseNetUnit3D(x, growth_rate=12, ksize=3, rep=6, name=name+'_denseblock1')

    out_15rf = BatchNormalization(center=True, scale=True)(x)
    out_15rf = Activation('relu')(out_15rf)
    out_15rf = Conv3DWithBN(out_15rf, filters=96, ksize=1, strides=1, name=name + '_out_15_postconv')

    x = DenseNetUnit3D(x, growth_rate=12, ksize=3, rep=6, name=name+'_denseblock2')

    out_27rf = BatchNormalization(center=True, scale=True)(x)
    out_27rf = Activation('relu')(out_27rf)
    out_27rf = Conv3DWithBN(out_27rf, filters=168, ksize=1, strides=1, name=name + '_out_27_postconv')

    return out_15rf, out_27rf

def BraTS2ScaleDenseNetConcat_large(input, name):

    x = Conv3D(filters=48, kernel_size=3, strides=1, padding='same', name=name+'_conv_init')(input)
    x = DenseNetUnit3D(x, growth_rate=12, ksize=3, rep=6, name=name+'_denseblock1')

    out_15rf = BatchNormalization(center=True, scale=True)(x)
    out_15rf = Activation('relu')(out_15rf)
    out_15rf = Conv3DWithBN(out_15rf, filters=192, ksize=1, strides=1, name=name + '_out_15_postconv')

    x = DenseNetUnit3D(x, growth_rate=24, ksize=3, rep=6, name=name+'_denseblock2')

    out_27rf = BatchNormalization(center=True, scale=True)(x)
    out_27rf = Activation('relu')(out_27rf)
    out_27rf = Conv3DWithBN(out_27rf, filters=336, ksize=1, strides=1, name=name + '_out_27_postconv')

    return out_15rf, out_27rf


def BraTS2ScaleDenseNet(input, num_labels):

    x = Conv3D(filters=24, kernel_size=3, strides=1, padding='same')(input)
    x = DenseNetUnit3D(x, growth_rate=12, ksize=3, rep=6)

    out_15rf = BatchNormalization(center=True, scale=True)(x)
    out_15rf = Activation('relu')(out_15rf)
    out_15rf = Conv3DWithBN(out_15rf, filters=96, ksize=1, strides=1, name='out_15_postconv')

    x = DenseNetUnit3D(x, growth_rate=12, ksize=3, rep=6)

    out_27rf = BatchNormalization(center=True, scale=True)(x)
    out_27rf = Activation('relu')(out_27rf)
    out_27rf = Conv3DWithBN(out_27rf, filters=168, ksize=1, strides=1, name='out_27_postconv')

    score_15rf = Conv3D(num_labels, kernel_size=1, strides=1, padding='same')(out_15rf)
    score_27rf = Conv3D(num_labels, kernel_size=1, strides=1, padding='same')(out_27rf)

    score = score_15rf[:, 13:25, 13:25, 13:25, :] + \
            score_27rf[:, 13:25, 13:25, 13:25, :]

    return score


def BraTS3ScaleDenseNet(input, num_labels):

    x = Conv3D(filters=24, kernel_size=3, strides=1, padding='same')(input)
    x = DenseNetUnit3D(x, growth_rate=12, ksize=3, rep=5)

    out_13rf = BatchNormalization(center=True, scale=True)(x)
    out_13rf = Activation('relu')(out_13rf)
    out_13rf = Conv3DWithBN(out_13rf, filters=84, ksize=1, strides=1, name='out_13_postconv')

    x = DenseNetUnit3D(x, growth_rate=12, ksize=3, rep=5)

    out_23rf = BatchNormalization(center=True, scale=True)(x)
    out_23rf = Activation('relu')(out_23rf)
    out_23rf = Conv3DWithBN(out_23rf, filters=144, ksize=1, strides=1, name='out_23_postconv')

    x = DenseNetUnit3D(x, growth_rate=12, ksize=3, rep=5)

    out_33rf = BatchNormalization(center=True, scale=True)(x)
    out_33rf = Activation('relu')(out_33rf)
    out_33rf = Conv3DWithBN(out_33rf, filters=204, ksize=1, strides=1, name='out_33_postconv')

    score_13rf = Conv3D(num_labels, kernel_size=1, strides=1, padding='same')(out_13rf)
    score_23rf = Conv3D(num_labels, kernel_size=1, strides=1, padding='same')(out_23rf)
    score_33rf = Conv3D(num_labels, kernel_size=1, strides=1, padding='same')(out_33rf)

    score = score_13rf[:, 16:28, 16:28, 16:28, :] + \
            score_23rf[:, 16:28, 16:28, 16:28, :] + \
            score_33rf[:, 16:28, 16:28, 16:28, :]

    return score



def BraTS1ScaleDenseNet(input, num_labels):

    x = Conv3D(filters=36, kernel_size=5, strides=1, padding='same')(input)
    x = DenseNetUnit3D(x, growth_rate=18, ksize=3, rep=6)

    out_15rf = BatchNormalization(center=True, scale=True)(x)
    out_15rf = Activation('relu')(out_15rf)
    out_15rf = Conv3DWithBN(out_15rf, filters=144, ksize=1, strides=1, name='out_17_postconv1')
    out_15rf = Conv3DWithBN(out_15rf, filters=144, ksize=1, strides=1, name='out_17_postconv2')

    score_15rf = Conv3D(num_labels, kernel_size=1, strides=1, padding='same')(out_15rf)

    score = score_15rf[:, 8:20, 8:20, 8:20, :]
    return score

In [13]:
files = '''BraTS19_2013_10_1/
BraTS19_2013_11_1/
BraTS19_2013_12_1/
BraTS19_2013_13_1/
BraTS19_2013_14_1/
BraTS19_2013_17_1/
BraTS19_2013_18_1/
BraTS19_2013_19_1/
BraTS19_2013_2_1/
BraTS19_2013_20_1/
BraTS19_2013_21_1/
BraTS19_2013_22_1/
BraTS19_2013_23_1/
BraTS19_2013_25_1/
BraTS19_2013_26_1/
BraTS19_2013_27_1/
BraTS19_2013_3_1/
BraTS19_2013_4_1/
BraTS19_2013_5_1/
BraTS19_2013_7_1/
BraTS19_CBICA_AAB_1/
BraTS19_CBICA_AAG_1/
BraTS19_CBICA_AAL_1/
BraTS19_CBICA_AAP_1/
BraTS19_CBICA_ABB_1/
BraTS19_CBICA_ABE_1/
BraTS19_CBICA_ABM_1/
BraTS19_CBICA_ABN_1/
BraTS19_CBICA_ABO_1/
BraTS19_CBICA_ABY_1/
BraTS19_CBICA_ALN_1/
BraTS19_CBICA_ALU_1/
BraTS19_CBICA_ALX_1/
BraTS19_CBICA_AME_1/
BraTS19_CBICA_AMH_1/
BraTS19_CBICA_ANG_1/
BraTS19_CBICA_ANI_1/
BraTS19_CBICA_ANP_1/
BraTS19_CBICA_ANV_1/
BraTS19_CBICA_ANZ_1/
BraTS19_CBICA_AOC_1/
BraTS19_CBICA_AOD_1/
BraTS19_CBICA_AOH_1/
BraTS19_CBICA_AOO_1/
BraTS19_CBICA_AOP_1/
BraTS19_CBICA_AOS_1/
BraTS19_CBICA_AOZ_1/
BraTS19_CBICA_APK_1/
BraTS19_CBICA_APR_1/
BraTS19_CBICA_APY_1/
BraTS19_CBICA_APZ_1/
BraTS19_CBICA_AQA_1/
BraTS19_CBICA_AQD_1/
BraTS19_CBICA_AQG_1/
BraTS19_CBICA_AQJ_1/
BraTS19_CBICA_AQN_1/
BraTS19_CBICA_AQO_1/
BraTS19_CBICA_AQP_1/
BraTS19_CBICA_AQQ_1/
BraTS19_CBICA_AQR_1/
BraTS19_CBICA_AQT_1/
BraTS19_CBICA_AQU_1/
BraTS19_CBICA_AQV_1/
BraTS19_CBICA_AQY_1/
BraTS19_CBICA_AQZ_1/
BraTS19_CBICA_ARF_1/
BraTS19_CBICA_ARW_1/
BraTS19_CBICA_ARZ_1/
BraTS19_CBICA_ASA_1/
BraTS19_CBICA_ASE_1/
BraTS19_CBICA_ASF_1/
BraTS19_CBICA_ASG_1/
BraTS19_CBICA_ASH_1/
BraTS19_CBICA_ASK_1/
BraTS19_CBICA_ASN_1/
BraTS19_CBICA_ASO_1/
BraTS19_CBICA_ASR_1/
BraTS19_CBICA_ASU_1/
BraTS19_CBICA_ASV_1/
BraTS19_CBICA_ASW_1/
BraTS19_CBICA_ASY_1/
BraTS19_CBICA_ATB_1/
BraTS19_CBICA_ATD_1/
BraTS19_CBICA_ATF_1/
BraTS19_CBICA_ATN_1/
BraTS19_CBICA_ATP_1/
BraTS19_CBICA_ATV_1/
BraTS19_CBICA_ATX_1/
BraTS19_CBICA_AUA_1/
BraTS19_CBICA_AUN_1/
BraTS19_CBICA_AUQ_1/
BraTS19_CBICA_AUR_1/
BraTS19_CBICA_AUW_1/
BraTS19_CBICA_AUX_1/
BraTS19_CBICA_AVB_1/
BraTS19_CBICA_AVF_1/
BraTS19_CBICA_AVG_1/
BraTS19_CBICA_AVJ_1/
BraTS19_CBICA_AVT_1/
BraTS19_CBICA_AVV_1/
BraTS19_CBICA_AWG_1/
BraTS19_CBICA_AWH_1/
BraTS19_CBICA_AWI_1/
BraTS19_CBICA_AWV_1/
BraTS19_CBICA_AWX_1/
BraTS19_CBICA_AXJ_1/
BraTS19_CBICA_AXL_1/
BraTS19_CBICA_AXM_1/
BraTS19_CBICA_AXN_1/
BraTS19_CBICA_AXO_1/
BraTS19_CBICA_AXQ_1/
BraTS19_CBICA_AXW_1/
BraTS19_CBICA_AYA_1/
BraTS19_CBICA_AYC_1/
BraTS19_CBICA_AYG_1/
BraTS19_CBICA_AYI_1/
BraTS19_CBICA_AYU_1/
BraTS19_CBICA_AYW_1/
BraTS19_CBICA_AZD_1/
BraTS19_CBICA_AZH_1/
BraTS19_CBICA_BAN_1/
BraTS19_CBICA_BAP_1/
BraTS19_CBICA_BAX_1/
BraTS19_CBICA_BBG_1/
BraTS19_CBICA_BCF_1/
BraTS19_CBICA_BCL_1/
BraTS19_CBICA_BDK_1/
BraTS19_CBICA_BEM_1/
BraTS19_CBICA_BFB_1/
BraTS19_CBICA_BFP_1/
BraTS19_CBICA_BGE_1/
BraTS19_CBICA_BGG_1/
BraTS19_CBICA_BGN_1/
BraTS19_CBICA_BGO_1/
BraTS19_CBICA_BGR_1/
BraTS19_CBICA_BGT_1/
BraTS19_CBICA_BGW_1/
BraTS19_CBICA_BGX_1/
BraTS19_CBICA_BHB_1/
BraTS19_CBICA_BHK_1/
BraTS19_CBICA_BHM_1/
BraTS19_CBICA_BHQ_1/
BraTS19_CBICA_BHV_1/
BraTS19_CBICA_BHZ_1/
BraTS19_CBICA_BIC_1/
BraTS19_CBICA_BJY_1/
BraTS19_CBICA_BKV_1/
BraTS19_CBICA_BLJ_1/
BraTS19_CBICA_BNR_1/
BraTS19_TCIA01_131_1/
BraTS19_TCIA01_147_1/
BraTS19_TCIA01_150_1/
BraTS19_TCIA01_180_1/
BraTS19_TCIA01_186_1/
BraTS19_TCIA01_190_1/
BraTS19_TCIA01_201_1/
BraTS19_TCIA01_203_1/
BraTS19_TCIA01_221_1/
BraTS19_TCIA01_231_1/
BraTS19_TCIA01_235_1/
BraTS19_TCIA01_335_1/
BraTS19_TCIA01_378_1/
BraTS19_TCIA01_390_1/
BraTS19_TCIA01_401_1/
BraTS19_TCIA01_411_1/
BraTS19_TCIA01_412_1/
BraTS19_TCIA01_425_1/
BraTS19_TCIA01_429_1/
BraTS19_TCIA01_448_1/
BraTS19_TCIA01_460_1/
BraTS19_TCIA01_499_1/
BraTS19_TCIA02_117_1/
BraTS19_TCIA02_118_1/
BraTS19_TCIA02_135_1/
BraTS19_TCIA02_151_1/
BraTS19_TCIA02_168_1/
BraTS19_TCIA02_171_1/
BraTS19_TCIA02_179_1/
BraTS19_TCIA02_198_1/
BraTS19_TCIA02_208_1/
BraTS19_TCIA02_222_1/
BraTS19_TCIA02_226_1/
BraTS19_TCIA02_274_1/
BraTS19_TCIA02_283_1/
BraTS19_TCIA02_290_1/
BraTS19_TCIA02_300_1/
BraTS19_TCIA02_309_1/
BraTS19_TCIA02_314_1/
BraTS19_TCIA02_321_1/
BraTS19_TCIA02_322_1/
BraTS19_TCIA02_331_1/
BraTS19_TCIA02_368_1/
BraTS19_TCIA02_370_1/
BraTS19_TCIA02_374_1/
BraTS19_TCIA02_377_1/
BraTS19_TCIA02_394_1/
BraTS19_TCIA02_430_1/
BraTS19_TCIA02_455_1/
BraTS19_TCIA02_471_1/
BraTS19_TCIA02_473_1/
BraTS19_TCIA02_491_1/
BraTS19_TCIA02_605_1/
BraTS19_TCIA02_606_1/
BraTS19_TCIA02_607_1/
BraTS19_TCIA02_608_1/
BraTS19_TCIA03_121_1/
BraTS19_TCIA03_133_1/
BraTS19_TCIA03_138_1/
BraTS19_TCIA03_199_1/
BraTS19_TCIA03_257_1/
BraTS19_TCIA03_265_1/
BraTS19_TCIA03_296_1/
BraTS19_TCIA03_338_1/
BraTS19_TCIA03_375_1/
BraTS19_TCIA03_419_1/
BraTS19_TCIA03_474_1/
BraTS19_TCIA03_498_1/
BraTS19_TCIA04_111_1/
BraTS19_TCIA04_149_1/
BraTS19_TCIA04_192_1/
BraTS19_TCIA04_328_1/
BraTS19_TCIA04_343_1/
BraTS19_TCIA04_361_1/
BraTS19_TCIA04_437_1/
BraTS19_TCIA04_479_1/
BraTS19_TCIA05_277_1/
BraTS19_TCIA05_396_1/
BraTS19_TCIA05_444_1/
BraTS19_TCIA05_478_1/
BraTS19_TCIA06_165_1/
BraTS19_TCIA06_184_1/
BraTS19_TCIA06_211_1/
BraTS19_TCIA06_247_1/
BraTS19_TCIA06_332_1/
BraTS19_TCIA06_372_1/
BraTS19_TCIA06_409_1/
BraTS19_TCIA06_603_1/
BraTS19_TCIA08_105_1/
BraTS19_TCIA08_113_1/
BraTS19_TCIA08_162_1/
BraTS19_TCIA08_167_1/
BraTS19_TCIA08_205_1/
BraTS19_TCIA08_218_1/
BraTS19_TCIA08_234_1/
BraTS19_TCIA08_242_1/
BraTS19_TCIA08_278_1/
BraTS19_TCIA08_280_1/
BraTS19_TCIA08_319_1/
BraTS19_TCIA08_406_1/
BraTS19_TCIA08_436_1/
BraTS19_TCIA08_469_1/
BraTS19_TMC_06290_1/
BraTS19_TMC_06643_1/
BraTS19_TMC_11964_1/
BraTS19_TMC_12866_1/
BraTS19_TMC_15477_1/
BraTS19_TMC_21360_1/
BraTS19_TMC_27374_1/
BraTS19_TMC_30014_1/'''

files = files.splitlines()
for f in range(len(files)):
  files[f] = files[f].strip(' /')
print(files)
print(files[2])
print(len(files))

['BraTS19_2013_10_1', 'BraTS19_2013_11_1', 'BraTS19_2013_12_1', 'BraTS19_2013_13_1', 'BraTS19_2013_14_1', 'BraTS19_2013_17_1', 'BraTS19_2013_18_1', 'BraTS19_2013_19_1', 'BraTS19_2013_2_1', 'BraTS19_2013_20_1', 'BraTS19_2013_21_1', 'BraTS19_2013_22_1', 'BraTS19_2013_23_1', 'BraTS19_2013_25_1', 'BraTS19_2013_26_1', 'BraTS19_2013_27_1', 'BraTS19_2013_3_1', 'BraTS19_2013_4_1', 'BraTS19_2013_5_1', 'BraTS19_2013_7_1', 'BraTS19_CBICA_AAB_1', 'BraTS19_CBICA_AAG_1', 'BraTS19_CBICA_AAL_1', 'BraTS19_CBICA_AAP_1', 'BraTS19_CBICA_ABB_1', 'BraTS19_CBICA_ABE_1', 'BraTS19_CBICA_ABM_1', 'BraTS19_CBICA_ABN_1', 'BraTS19_CBICA_ABO_1', 'BraTS19_CBICA_ABY_1', 'BraTS19_CBICA_ALN_1', 'BraTS19_CBICA_ALU_1', 'BraTS19_CBICA_ALX_1', 'BraTS19_CBICA_AME_1', 'BraTS19_CBICA_AMH_1', 'BraTS19_CBICA_ANG_1', 'BraTS19_CBICA_ANI_1', 'BraTS19_CBICA_ANP_1', 'BraTS19_CBICA_ANV_1', 'BraTS19_CBICA_ANZ_1', 'BraTS19_CBICA_AOC_1', 'BraTS19_CBICA_AOD_1', 'BraTS19_CBICA_AOH_1', 'BraTS19_CBICA_AOO_1', 'BraTS19_CBICA_AOP_1', 'BraTS19_

In [0]:
import glob

import numpy as np
from keras.backend import learning_phase
from keras.layers import concatenate, Conv3D

#import tf_models
from sklearn.preprocessing import scale
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# from tensorflow.contrib.keras.python.keras.backend import learning_phase
# from tensorflow.contrib.keras.python.keras.layers import concatenate, Conv3D
from nibabel import load as load_nii
import os
#import argparse
import keras

'''
def parse_inputs():

    parser = argparse.ArgumentParser(description='train the model')
    parser.add_argument('-r', '--root-path', dest='root_path', default='/Users/amaeda/Documents/workspace_maeda/MRI-tumor-segmentation-Brats/data/BRATS2015_Training/HGG')
    parser.add_argument('-sp', '--save-path', dest='save_path', default='dense24_correction')
    parser.add_argument('-lp', '--load-path', dest='load_path', default='dense24_correction')
    parser.add_argument('-ow', '--offset-width', dest='offset_w', type=int, default=12)
    parser.add_argument('-oh', '--offset-height', dest='offset_h', type=int, default=12)
    parser.add_argument('-oc', '--offset-channel', dest='offset_c', nargs='+', type=int, default=12)
    parser.add_argument('-ws', '--width-size', dest='wsize', type=int, default=38)
    parser.add_argument('-hs', '--height-size', dest='hsize', type=int, default=38)
    parser.add_argument('-cs', '--channel-size', dest='csize', type=int, default=38)
    parser.add_argument('-ps', '--pred-size', dest='psize', type=int, default=12)
    parser.add_argument('-bs', '--batch-size', dest='batch_size', type=int, default=2)
    parser.add_argument('-e', '--num-epochs', dest='num_epochs', type=int, default=5)
    parser.add_argument('-c', '--continue-training', dest='continue_training', type=bool, default=False)
    parser.add_argument('-mn', '--model_name', dest='model_name', type=str, default='dense24')
    parser.add_argument('-nc', '--n4correction', dest='correction', type=bool, default=False)
    parser.add_argument('-gpu', '--gpu_id', dest='gpu_id', type=str, default='0')
    return vars(parser.parse_args())

options = parse_inputs()
'''

options = {}
options['root_path'] = brats2019_path + '/HGG/'
options['save_path'] = 'dense24_correction'
options['load_path'] = 'dense24_correction'
options['offset_w'] = 12
options['offset_h'] = 12
options['offset_c'] = 12
options['wsize'] = 38
options['hsize'] = 38
options['csize'] = 38
options['psize'] = 12
options['batch_size'] = 2
options['num_epochs'] = 5
options['continue_training'] = False
options['model_name'] = 'dense24'
options['correction'] = False
options['gpu_id'] = '0'

os.environ["CUDA_VISIBLE_DEVICES"] = options['gpu_id']
def acc_tf(y_pred, y_true):
    correct_prediction = tf.equal(tf.cast(tf.argmax(y_pred, -1), tf.int32), tf.cast(tf.argmax(y_true, -1), tf.int32))
    return 100 * tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


def get_patches_3d(data, labels, centers, hsize, wsize, csize, psize, preprocess=True):
    """
    :param data: 4D nparray (h, w, c, ?)
    :param centers:
    :param hsize:
    :param wsize:
    :param csize:
    :return:
    """
    patches_x, patches_y = [], []
    offset_p = (hsize - psize) / 2
    for i in range(len(centers[0])):
        h, w, c = centers[0, i], centers[1, i], centers[2, i]
        h_beg = int(min(max(0, h - hsize / 2), 240 - hsize))
        w_beg = int(min(max(0, w - wsize / 2), 240 - wsize))
        c_beg = int(min(max(0, c - csize / 2), 155 - csize))
        ph_beg = int(h_beg + offset_p)
        pw_beg = int(w_beg + offset_p)
        pc_beg = int(c_beg + offset_p)
        vox = data[h_beg:h_beg + hsize, w_beg:w_beg + wsize, c_beg:c_beg + csize, :]
        vox_labels = labels[ph_beg:ph_beg + psize, pw_beg:pw_beg + psize, pc_beg:pc_beg + psize]
        patches_x.append(vox)
        patches_y.append(vox_labels)
    return np.array(patches_x), np.array(patches_y)


def positive_ratio(x):
    return float(np.sum(np.greater(x, 0))) / np.prod(x.shape)


def norm(image):
    image = np.squeeze(image)
    image_nonzero = image[np.nonzero(image)]
    return (image - image_nonzero.mean()) / image_nonzero.std()


def segmentation_loss(y_true, y_pred, n_classes):
    y_true = tf.reshape(y_true, (-1, n_classes))
    y_pred = tf.reshape(y_pred, (-1, n_classes))
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,
                                                                  logits=y_pred))


def vox_preprocess(vox):
    vox_shape = vox.shape
    vox = np.reshape(vox, (-1, vox_shape[-1]))
    vox = scale(vox, axis=0)
    return np.reshape(vox, vox_shape)


def one_hot(y, num_classes):
    y_ = np.zeros([len(y), num_classes])
    y_[np.arange(len(y)), y] = 1
    return y_


def dice_coef_np(y_true, y_pred, num_classes):
    """
    :param y_true: sparse labels
    :param y_pred: sparse labels
    :param num_classes: number of classes
    :return:
    """
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    y_true = y_true.flatten()
    y_true = one_hot(y_true, num_classes)
    y_pred = y_pred.flatten()
    y_pred = one_hot(y_pred, num_classes)
    intersection = np.sum(y_true * y_pred, axis=0)
    return (2. * intersection) / (np.sum(y_true, axis=0) + np.sum(y_pred, axis=0))


def vox_generator(all_files, n_pos, n_neg,correction= False):
    path = options['root_path']

    def get_filename(prefix):
        return glob.glob(os.path.join(path, file, prefix))[0].split('/')[-1]

    while 1:
        for file in all_files:
            if correction:
                flair = load_nii(os.path.join(path, file, file + '_flair_corrected.nii.gz')).get_data()
                t2 = load_nii(os.path.join(path, file, file + '_t2_corrected.nii.gz')).get_data()
                t1 = load_nii(os.path.join(path, file, file + '_t1_corrected.nii.gz')).get_data()
                t1ce = load_nii(os.path.join(path, file, file + '_t1ce_corrected.nii.gz')).get_data()
            else:
                flair = load_nii(os.path.join(path, file, file + '_flair.nii.gz')).get_data()
                t2 = load_nii(os.path.join(path, file, file + '_t2.nii.gz')).get_data()
                t1 = load_nii(os.path.join(path, file, file + '_t1.nii.gz')).get_data()
                t1ce = load_nii(os.path.join(path, file, file + '_t1ce.nii.gz')).get_data()

            data_norm = np.array([norm(flair), norm(t2), norm(t1), norm(t1ce)])
            data_norm = np.transpose(data_norm, axes=[1, 2, 3, 0])
            labels = load_nii(os.path.join(path, file, file+'_seg.nii.gz')).get_data()

            foreground = np.array(np.where(labels > 0))
            background = np.array(np.where((labels == 0) & (flair > 0)))

            # n_pos = int(foreground.shape[1] * discount)
            foreground = foreground[:, np.random.permutation(foreground.shape[1])[:n_pos]]
            background = background[:, np.random.permutation(background.shape[1])[:n_neg]]

            centers = np.concatenate((foreground, background), axis=1)
            centers = centers[:, np.random.permutation(n_neg+n_pos)]

            yield data_norm, labels, centers


def label_transform(y, nlabels):
    return [
            keras.utils.to_categorical(np.copy(y).astype(dtype=np.bool),
                                       num_classes=2).reshape([y.shape[0], y.shape[1], y.shape[2], y.shape[3], 2]),

            keras.utils.to_categorical(y,
                                       num_classes=nlabels).reshape([y.shape[0], y.shape[1], y.shape[2], y.shape[3], nlabels])
            ]


def train():
    NUM_EPOCHS = options['num_epochs']
    LOAD_PATH = options['load_path']
    SAVE_PATH = options['save_path']
    PSIZE = options['psize']
    HSIZE = options['hsize']
    WSIZE = options['wsize']
    CSIZE = options['csize']
    model_name= options['model_name']
    BATCH_SIZE = options['batch_size']
    continue_training = options['continue_training']

    num_labels = 5
    '''
    files = []
    with open('train.txt') as f:
        for line in f:
            files.append(line[:-1])
    '''
    print('%d training samples' % len(files))

    # create training and testing vars
    files, val_files = train_test_split(files, test_size=0.2)
    print('%d training   samples' % len(files))
    print('%d validation samples' % len(val_files))

    flair_t2_node = tf.placeholder(dtype=tf.float32, shape=(None, HSIZE, WSIZE, CSIZE, 2))
    t1_t1ce_node = tf.placeholder(dtype=tf.float32, shape=(None, HSIZE, WSIZE, CSIZE, 2))
    flair_t2_gt_node = tf.placeholder(dtype=tf.int32, shape=(None, PSIZE, PSIZE, PSIZE, 2))
    t1_t1ce_gt_node = tf.placeholder(dtype=tf.int32, shape=(None, PSIZE, PSIZE, PSIZE, 5))

    if model_name == 'dense48':
        flair_t2_15, flair_t2_27 = BraTS2ScaleDenseNetConcat_large(input=flair_t2_node, name='flair')
        t1_t1ce_15, t1_t1ce_27 = BraTS2ScaleDenseNetConcat_large(input=t1_t1ce_node, name='t1')
    elif model_name == 'no_dense':

        flair_t2_15, flair_t2_27 = PlainCounterpart(input=flair_t2_node, name='flair')
        t1_t1ce_15, t1_t1ce_27 = PlainCounterpart(input=t1_t1ce_node, name='t1')

    elif model_name == 'dense24':

        flair_t2_15, flair_t2_27 = BraTS2ScaleDenseNetConcat(input=flair_t2_node, name='flair')
        t1_t1ce_15, t1_t1ce_27 = BraTS2ScaleDenseNetConcat(input=t1_t1ce_node, name='t1')
    else:
        print(' No such model name ')

    t1_t1ce_15 = concatenate([t1_t1ce_15, flair_t2_15])
    t1_t1ce_27 = concatenate([t1_t1ce_27, flair_t2_27])

    flair_t2_15 = Conv3D(2, kernel_size=1, strides=1, padding='same', name='flair_t2_15_cls')(flair_t2_15)
    flair_t2_27 = Conv3D(2, kernel_size=1, strides=1, padding='same', name='flair_t2_27_cls')(flair_t2_27)
    t1_t1ce_15 = Conv3D(num_labels, kernel_size=1, strides=1, padding='same', name='t1_t1ce_15_cls')(t1_t1ce_15)
    t1_t1ce_27 = Conv3D(num_labels, kernel_size=1, strides=1, padding='same', name='t1_t1ce_27_cls')(t1_t1ce_27)

    flair_t2_score = flair_t2_15[:, 13:25, 13:25, 13:25, :] + \
                     flair_t2_27[:, 13:25, 13:25, 13:25, :]

    t1_t1ce_score = t1_t1ce_15[:, 13:25, 13:25, 13:25, :] + \
                    t1_t1ce_27[:, 13:25, 13:25, 13:25, :]

    loss = segmentation_loss(flair_t2_gt_node, flair_t2_score, 2) + \
           segmentation_loss(t1_t1ce_gt_node, t1_t1ce_score, 5)

    acc_flair_t2 = acc_tf(y_pred=flair_t2_score, y_true=flair_t2_gt_node)
    acc_t1_t1ce = acc_tf(y_pred=t1_t1ce_score, y_true=t1_t1ce_gt_node)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        optimizer = tf.train.AdamOptimizer(learning_rate=5e-4).minimize(loss)

    saver = tf.train.Saver(max_to_keep=15)
    data_gen_train = vox_generator(all_files=files, n_pos=200, n_neg=200,correction = options['correction'])

    with tf.Session() as sess:
        if continue_training:
            saver.restore(sess, LOAD_PATH)
        else:
            sess.run(tf.global_variables_initializer())
        for ei in range(NUM_EPOCHS):
            for pi in range(len(files)):
                acc_pi, loss_pi = [], []
                data, labels, centers = data_gen_train.__next__()
                n_batches = int(np.ceil(float(centers.shape[1]) / BATCH_SIZE))
                for nb in range(n_batches):
                    offset_batch = min(nb * BATCH_SIZE, centers.shape[1] - BATCH_SIZE)
                    data_batch, label_batch = get_patches_3d(data, labels, centers[:, offset_batch:offset_batch + BATCH_SIZE], HSIZE, WSIZE, CSIZE, PSIZE, False)
                    label_batch = label_transform(label_batch, 5)
                    _, l, acc_ft, acc_t1c = sess.run(fetches=[optimizer, loss, acc_flair_t2, acc_t1_t1ce],
                                                   feed_dict={flair_t2_node: data_batch[:, :, :, :, :2],
                                                              t1_t1ce_node: data_batch[:, :, :, :, 2:],
                                                              flair_t2_gt_node: label_batch[0],
                                                              t1_t1ce_gt_node: label_batch[1],
                                                              learning_phase(): 1})
                    acc_pi.append([acc_ft, acc_t1c])
                    loss_pi.append(l)
                    n_pos_sum = np.sum(np.reshape(label_batch[0], (-1, 2)), axis=0)
                    print(f"epoch-patient: {ei + 1}, {pi + 1}, iter: {nb + 1}-{n_batches}, p%%: {n_pos_sum[1]/float(np.sum(n_pos_sum))}, loss: {l}, acc_flair_t2: {acc_ft}, acc_t1_t1ce: {acc_t1c}")

                print('patient loss: %.4f, patient acc: %.4f' % (np.mean(loss_pi), np.mean(acc_pi)))

            saver.save(sess, SAVE_PATH, global_step=ei)
            print('model saved')


if __name__ == '__main__':
    
    train()

259 training samples
epoch-patient: 1, 1, iter: 1-200, p%%: 0.33912037037037035, loss: 3.833038806915283, acc_flair_t2: 52.74884033203125, acc_t1_t1ce: 7.754630088806152
epoch-patient: 1, 1, iter: 2-200, p%%: 0.42650462962962965, loss: 2.7425498962402344, acc_flair_t2: 55.23727035522461, acc_t1_t1ce: 24.537036895751953
epoch-patient: 1, 1, iter: 3-200, p%%: 0.9091435185185185, loss: 1.281793236732483, acc_flair_t2: 69.53125, acc_t1_t1ce: 56.36574172973633
epoch-patient: 1, 1, iter: 4-200, p%%: 0.38570601851851855, loss: 1.6225559711456299, acc_flair_t2: 48.55324172973633, acc_t1_t1ce: 66.72454071044922
epoch-patient: 1, 1, iter: 5-200, p%%: 0.5, loss: 0.9159183502197266, acc_flair_t2: 89.61226654052734, acc_t1_t1ce: 82.00231170654297
epoch-patient: 1, 1, iter: 6-200, p%%: 0.1261574074074074, loss: 1.7381147146224976, acc_flair_t2: 87.84722137451172, acc_t1_t1ce: 48.061344146728516
epoch-patient: 1, 1, iter: 7-200, p%%: 0.0, loss: 3.6828441619873047, acc_flair_t2: 50.34722137451172, acc